# Web interface `s4refsimtool`
> Interactive web-based user interface for the CMB-S4 reference simulation tool

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
w = {}

In [ ]:
for emission in ["foreground_emission", "CMB_unlensed", "CMB_lensing_signal"]:
    w[emission] = widgets.BoundedFloatText(
    value=1,
    min=0,
    max=1,
    step=0.01,
    description=f'{emission}:',
        disabled=False
    )

In [ ]:
emission = "CMB_tensor_to_scalar_ratio"
w[emission] = widgets.BoundedFloatText(
    value=3e-3,
    min=0,
    max=1,
    step=1e-5,
    description=f'{emission}:',
        disabled=False
    )


## Sky emission weighting

Each sky emission has a weighting factor between 0 and 1

### Foreground emission

Synchrotron, Dust, Free-free, AME
Websky CIB, tSZ, kSZ

In [ ]:
display(w["foreground_emission"])

### Unlensed CMB
Planck cosmological parameters, no tensor modes

In [ ]:
display(w["CMB_unlensed"])

### CMB lensing signal

CMB lensed - CMB unlensed:
* 1 for lensed CMB
* 0 for unlensed CMB
* `>0, <1` for residual after de-lensing

In [ ]:
display(w["CMB_lensing_signal"])

### CMB tensor to scalar ratio

Value of the `r` cosmological parameter

In [ ]:
display(w["CMB_tensor_to_scalar_ratio"])

## Experiment parameters

### Total experiment length

In years, supports decimals

In [ ]:
w["total_experiment_length_years"] = widgets.BoundedFloatText(
    value=7,
    min=0,
    max=15,
    step=0.1,
    description='Years:',
        disabled=False
    )

In [ ]:
display(w["total_experiment_length_years"])

In [ ]:
w["observing_efficiency"] = widgets.BoundedFloatText(
    value=0.2,
    min=0,
    max=1,
    step=0.01,
    description='Efficiency:',
        disabled=False
    )

### Observing efficiency

Typically 20%, use decimal notation

In [ ]:
display(w["observing_efficiency"])

In [ ]:
w["number_of_splits"] = widgets.BoundedFloatText(
    value=1,
    min=1,
    max=7,
    step=1,
    description='Splits:',
        disabled=False
    )

### Number of splits

Number of splits, 1 generates only full mission
2-7 generates the full mission map and then the requested number
of splits scaled accordingly. E.g. 7 generates the full mission
map and 7 equal (yearly) maps

In [ ]:
display(w["number_of_splits"])

## Telescope configuration

Currently we constraint to have a total of 6 SAT and 3 LAT,
each SAT has a maximum of 3 tubes, each LAT of 19.
The checkbox on the right of each telescope checks that the amount of number of tubes is correct.

In [ ]:
import toml
config = toml.load("s4_reference_design.toml")

In [ ]:
def define_check_sum(telescope_widgets, max_tubes):
    def check_sum(_):
        total_tubes = sum([w.value for w in telescope_widgets[1:1+4]])
        telescope_widgets[0].value = total_tubes == max_tubes
    return check_sum

In [ ]:
telescopes = {"SAT":{}, "LAT":{}}
for telescope, tubes in config["telescopes"]["SAT"].items():
    telescopes["SAT"][telescope] = [widgets.Valid(
    value=True, description=telescope, layout=widgets.Layout(width='120px')
    )]
    telescope_check = define_check_sum(telescopes["SAT"][telescope], 3)
    for k,v in tubes.items():
        if k == "site":
            wid = widgets.Dropdown(
            options=['Pole', 'Chile'],
            value=v,
            description=k,
            disabled=False, layout=widgets.Layout(width='200px')
            )
        elif k == "years":
            wid = widgets.BoundedFloatText(
    value=v,
    min=0,
    max=20,
    step=0.1,
    description='years',
        disabled=False, layout=widgets.Layout(width='120px')
    )
        else:
            
            wid = widgets.BoundedIntText(
    value=v,
    min=0,
    max=3,
    step=1,
    description=k,
    disabled=False, layout=widgets.Layout(width='120px')
            ) 
            wid.observe(telescope_check)
        telescopes["SAT"][telescope].append(wid)

In [ ]:
for k, v in telescopes["SAT"].items():
    display(widgets.HBox(v))

In [ ]:
for telescope, tubes in config["telescopes"]["LAT"].items():
    telescopes["LAT"][telescope] = [widgets.Valid(
    value=True, description=telescope, layout=widgets.Layout(width='120px')
    )]
    telescope_check = define_check_sum(telescopes["LAT"][telescope], 19)
    for k,v in tubes.items():
        if k == "site":
            wid = widgets.Dropdown(
            options=['Pole', 'Chile'],
            value=v,
            description=k,
            disabled=False, layout=widgets.Layout(width='200px')
            )
        elif k == "years":
            wid = widgets.BoundedFloatText(
    value=v,
    min=0,
    max=20,
    step=0.1,
    description='years',
        disabled=False, layout=widgets.Layout(width='130px')
    )
        else:
            
            wid = widgets.BoundedIntText(
    value=v,
    min=0,
    max=19,
    step=1,
    description=k,
    disabled=False, layout=widgets.Layout(width='130px')
            ) 
            wid.observe(telescope_check)
        telescopes["LAT"][telescope].append(wid)

In [ ]:
for k, v in telescopes["LAT"].items():
    display(widgets.HBox(v))

In [ ]:
import toml

In [ ]:
def generate_toml():
    output_config = {}
    for section in ["sky_emission", "experiment"]:
        output_config[section] = {}
        for k in config[section]:
            output_config[section][k] = w[k].value
    output_config["telescopes"] = {"SAT":{}, "LAT":{}}
    for t in ["SAT", "LAT"]:
        for telescope, tubes in telescopes[t].items():
            output_config["telescopes"][t][telescope] = toml._get_empty_inline_table(dict)
            for tube_type in tubes[1:]:
                output_config["telescopes"][t][telescope][tube_type.description] = tube_type.value
                if tube_type.description == "years":
                    output_config["telescopes"][t][telescope][tube_type.description] = int(tube_type.value)
    return toml.dumps(output_config, preserve=True)

## Generate a TOML configuration file

Click on the button to generate the TOML file and display it.

In [ ]:
button = widgets.Button(
    description='Generate TOML',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
output = widgets.Output(layout={'border': '1px solid black'})

display(button, output)

def on_button_clicked(b):
    output.clear_output()
    with output:
        print(generate_toml())

button.on_click(on_button_clicked)